# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

In [1]:
# Holdout validation is another name for train-test split

# Before starting, make sure I'm in the learn-env kernel because
# it has more up-to-date version of sklearn

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [3]:
boston_features.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [4]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['TARGET'])

In [5]:
X.head()

,CHAS,RM,DIS,B,LSTAT
0,0.0,6.575,0.542096,1.000000,-1.275260
1,0.0,6.421,0.623954,1.000000,-0.263711
2,0.0,7.185,0.623954,0.989737,-1.627858
3,0.0,6.998,0.707895,0.994276,-2.153192
4,0.0,7.147,0.707895,1.000000,-1.162114


In [6]:
y.head()

,TARGET
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


## Train test split

Perform a train-test-split with a test set of 0.20.

In [7]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2)

In [20]:
# I'll take a look at the lengths just to see if it makes sense.
# The test sizes should be 20% of the combined train+test size.

len(X_train), len(X_test), len(Y_train), len(Y_test)

(404, 102, 404, 102)

Fit the model and apply the model to the make test set predictions

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
# Now I'll make an instance of the linear model

linreg = LinearRegression()

In [23]:
# Now I'll fit my model with the training data

linreg.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Calculate the residuals and the mean squared error

In [24]:
# In order to do this, I need a predicted value. That'll be y_hat_test.

y_hat_test = linreg.predict(X_test)

In [25]:
test_residuals = y_hat_test - Y_test

In [26]:
# Now take the mean squared error (mse)

(test_residuals**2).mean()

TARGET    23.900879
dtype: float64

In [27]:
# Another way to obtain the mse...

from sklearn.metrics import mean_squared_error

In [28]:
test_mse = mean_squared_error(Y_test, y_hat_test)
test_mse

23.90087864361238

In [29]:
# Now, as a sanity check, let's compare the mse of the testing data to the
# mse of the training data

y_hat_train = linreg.predict(X_train)
train_mse = mean_squared_error(Y_train, y_hat_train)
train_mse

# Not too far off! However, note that it will be different every time with train-test splits.
# Expect to get much better results with cross-validation.

21.292792747199282

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [30]:
# There's a numpy function that will do this for us.

import numpy as np

In [34]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers           
    return np.array_split(data, k)

In [31]:
# First, I need to concatanae my X and Y to make one DF.

df = pd.concat([X, y], axis = 1)

In [32]:
df.head()

,CHAS,RM,DIS,B,LSTAT,TARGET
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [37]:
# The instructions at the top of the notebook say to construct a 5-fold cross validation.

data_folds = kfolds(df, 5)

In [38]:
# sanity check

len(data_folds)

5

### Apply it to the Boston Housing Data

In [ ]:
# Make sure to concatenate the data again

In [39]:
# I already did this above

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [40]:
# I want fold 0 to be my testing data
# I want folds 1 through 4 to be my training data

n = 0
train_list = []
for counts, fold in enumerate(data_folds):
    if counts != n:
        train_list.append(fold)
df_train = pd.concat(train_list)
df_test = data_folds[n]

In [42]:
df_train.shape, df_test.shape

((404, 6), (102, 6))

In [ ]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train_list = []
    for counts, fold in enumerate(data_folds):
        if counts != n:
            train_list.append(fold)
    train = df.concat(train_list)
    test = data_folds[n]
    # Fit a linear regression model
    linreg.fit(train, test)
    #Evaluate Train and Test Errors

# print(train_errs)
# print(test_errs)

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!